In [ ]:
%matplotlib inline
import shutil
from io import StringIO
from IPython.display import display, Math, Latex
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})
import os 

In [ ]:
WORKDIR = '../'

# Dictionary, atom and system properties
d = {
'Forcefield name': 'ff_our',
'Water model': 'spce',
'Box size': 60,
'Input concentrations':    [0.80, 1.39, 1.84], # NaCl: [0.82, 1.45, 1.935], NaI: [0.80, 1.39, 1.84]
'Molal concentrations':    [1.0, 2.0, 3.0],  
'Bond length':             {'OH':0.10000}, 
'Harmonic bond constant':  {'OH':345000}, 
'Bond angle':              {'HOH':1.91061193216}, 
'Harmonic angle constant': {'HOH':383}, 
'Mass':           {'Na':22.9898, 'Cl':35.453, 'I':126.90 },
'Partial charge': {'O':-0.8476,  'H':0.4238,  'Na':1.000, 'Cl':-1.000,  'I':-1.000 },  
'Size':           {'O':0.3166,   'H':0,       'Na':0.255, 'Cl':0.43900, 'I':0.491  },  
'Well depth':     {'O':0.650,    'H':0,       'Na':0.28,  'Cl':0.41600, 'I':0.158  },
'Steps':          {'Simulation':20000000, 'Report':1000}, # 20000000
'Constants':      {'A':6.022*pow(10,23),  'Unit converter':pow(10,-9)}
}

cation = 'Na'
cation_name = 'sodium'

anion = 'I'
anion_name = 'iodide'

pot = 'SP3'

cutoff_value = 1.28 # pair potential cutoff

def conc2num(conc,box_size):
    n_anion = conc*pow(box_size*d['Constants']['Unit converter'],3)*d['Constants']['A']
    return round(n_anion)

In [ ]:
print('Current working directory: ', os.getcwd())
for (conc_input, conc_m) in zip(d['Input concentrations'], d['Molal concentrations']):
    
    wdir = WORKDIR+'data/'+pot+'/'+str(cutoff_value)+'nm/'+cation.lower()+anion.lower()+'/'+str(conc_m)+'m/'
    
    # write input file for packmol
    
    print('Concentration input:', conc_input, ' M results in', conc2num(conc_input, d['Box size']), anion+'-ions')

    PACKMOL_INPUT = """ 
# Mixture 

tolerance %f
filetype pdb
output %s
add_amber_ter

structure %s
  number %d 
  inside box 0. 0. 0. %f %f %f
end structure
"""

    PACKMOL_INPUT = PACKMOL_INPUT % (1,str(conc_m)+'m_box.pdb',str(conc_m)+'m_'+anion.lower()+'.pdb',conc2num(conc_input, d['Box size']),d['Box size'],d['Box size'],d['Box size'])
    file_handle = open('packmol_input_'+anion.lower()+'.txt', 'w')
    file_handle.write(PACKMOL_INPUT)
    file_handle.close()
    
    # write pdb file for single anion

    anion_pdb = """CRYST1  %f  %f  %f  90.00  90.00  90.00 P 1           1
HETATM    1  %s  IN      1      20.000  20.000  20.000  1.00  0.00          %s
END"""

    anion_pdb = anion_pdb % (d['Box size'], d['Box size'], d['Box size'], anion, anion)
    
    with open(str(conc_m)+'m_'+anion.lower()+'.pdb', 'w') as text_file:
        text_file.write(anion_pdb)

    # use packmol to create a system of n_anion randomly placed anions
    os.system("%s < %s" % ('packmol', 'packmol_input_'+anion.lower()+'.txt'))
    
    # move created files to the dedicated folder
    for file in [str(conc_m)+'m_'+anion.lower()+'.pdb', str(conc_m)+'m_box.pdb', 'packmol_input_'+anion.lower()+'.txt']:
        shutil.move(os.path.join('./', file), os.path.join(wdir, file))

In [ ]:
def create_ff_file(alt):
    FFXML_topology = """<ForceField>

     <AtomTypes>

      <Type name="%s" class="%s" element="%s" mass="%f"/>
      <Type name="%s" class="%s" element="%s" mass="%f"/>
      <Type name="spce-O" class="OW" element="O" mass="15.99943"/>
      <Type name="spce-H" class="HW" element="H" mass="1.007947"/>

     </AtomTypes>

     <Residues>

      <Residue name="HOH">
       <Atom name="O" type="spce-O"/>
       <Atom name="H1" type="spce-H"/>
       <Atom name="H2" type="spce-H"/>
       <Bond from="0" to="1"/>
       <Bond from="0" to="2"/>
      </Residue>

      <Residue name="%s">
       <Atom name="%s" type="%s"/>
      </Residue>

      <Residue name="%s">
       <Atom name="%s" type="%s"/>
      </Residue>

     </Residues>"""

    FFXML_bonded = """
     <HarmonicBondForce>
      <Bond class1="OW" class2="HW" length="%f" k="%f"/>
     </HarmonicBondForce>

     <HarmonicAngleForce>    
      <Angle class1="HW" class2="OW" class3="HW" angle="%f" k="%f"/>
     </HarmonicAngleForce>"""

    if alt is 'nonbonded':
        FFXML_nonbonded = """
     <NonbondedForce coulomb14scale="0.5" lj14scale="0.5">
      <Atom type="spce-O" charge="%f" sigma="%f" epsilon="%f"/>
      <Atom type="spce-H" charge="%f" sigma="%f" epsilon="%f"/>
      <Atom type="%s" charge="%f" sigma="%f" epsilon="%f"/>
      <Atom type="%s" charge="%f" sigma="%f" epsilon="%f"/>
     </NonbondedForce>
    </ForceField>"""
        
    elif alt is 'nonbonded_custom':
        FFXML_nonbonded = """
     <CustomNonbondedForce energy="to be defined..." bondCutoff="3">
      <GlobalParameter name="alpha" defaultValue="1.0"/>
      <PerParticleParameter name="charge"/>
      <PerParticleParameter name="sigma"/>
      <PerParticleParameter name="epsilon"/>
      <Atom type="spce-O" charge="%f" sigma="%f" epsilon="%f"/>
      <Atom type="spce-H" charge="%f" sigma="%f" epsilon="%f"/>
      <Atom type="%s" charge="%f" sigma="%f" epsilon="%f"/>
      <Atom type="%s" charge="%f" sigma="%f" epsilon="%f"/>
     </CustomNonbondedForce>
    </ForceField>"""


    FFXML_topology = FFXML_topology % (cation_name, cation, cation, d['Mass'][cation], anion_name, anion, anion, d['Mass'][anion], cation, cation, cation_name, anion, anion, anion_name)

    FFXML_bonded = FFXML_bonded % (d['Bond length']['OH'], d['Harmonic bond constant']['OH'], d['Bond angle']['HOH'], d['Harmonic angle constant']['HOH'])

    FFXML_nonbonded = FFXML_nonbonded % (d['Partial charge']['O'], d['Size']['O'], d['Well depth']['O'], d['Partial charge']['H'], d['Size']['H'], d['Well depth']['H'], cation_name, d['Partial charge'][cation], d['Size'][cation], d['Well depth'][cation], anion_name, d['Partial charge'][anion], d['Size'][anion], d['Well depth'][anion])

    FFXML = FFXML_topology + FFXML_bonded + FFXML_nonbonded


    for conc_m in d['Molal concentrations']:
        wdir = WORKDIR+'data/'+pot+'/'+str(cutoff_value)+'nm/'+cation.lower()+anion.lower()+'/'+str(conc_m)+'m/'
        if alt is 'nonbonded':
            with open(wdir+'nonbonded.xml', 'w') as text_file:
                text_file.write(FFXML)
        elif alt is 'nonbonded_custom':
            with open(wdir+'nonbonded_custom.xml', 'w') as text_file:
                text_file.write(FFXML)

    print(FFXML)

In [ ]:
create_ff_file('nonbonded')
create_ff_file('nonbonded_custom')

In [ ]:
for conc_m in d['Molal concentrations']:
    openmm_script="""
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
import numpy as np

def findForce(system, forcetype, add=True):
# Finds a specific force in the system force list - added if not found
    for force in system.getForces():
        if isinstance(force, forcetype):
            return force
    if add==True:
        system.addForce(forcetype())
        return findForce(system, forcetype)
    return None

pdb = PDBFile("%s")
PDBFile.writeFile(pdb.topology, pdb.positions, open("%s", 'w'))

ff = ForceField("%s") # this will create a NonbondedForce

modeller = Modeller(pdb.topology, pdb.positions)
# Add  water and cations
modeller.addSolvent(ff,model='spce',boxSize=(%f,%f,%f)*angstroms,positiveIon='%s')

# Create the OpenMM system
print('Creating OpenMM System')

system = ff.createSystem(
    modeller.topology, nonbondedMethod=PME,
    nonbondedCutoff=%f*nanometers, constraints=AllBonds, rigidWater=True, ewaldErrorTolerance=0.0005)

nonbonded = findForce(system, NonbondedForce)
nonbonded.setUseDispersionCorrection( True )

# Create the integrator to do Langevin dynamics
integrator = LangevinIntegrator(
                        298.15*kelvin,       # Temperature of heat bath
                        1.0/picoseconds,  # Friction coefficient
                        2.0*femtoseconds, # Time step
)
integrator.setConstraintTolerance(0.00001)

# NPT ensemble
barostat = MonteCarloBarostat(1.0*bar, 298.15*kelvin, 25) 
system.addForce(barostat)
# Define the platform to use; CUDA, OpenCL, CPU, or Reference. Or do not specify
# the platform to use the default (fastest) platform
platform = Platform.getPlatformByName('CUDA') # change to what you like: CUDA, CPU etc...

#prop = dict(CudaPrecision='mixed',CudaDeviceIndex='0,1') # comment out this line if running on CPU

# Create the Simulation object
sim = Simulation(modeller.topology, system, integrator, platform)

#print(platform.getPropertyValue(sim.context,'CudaDeviceIndex')) # comment out this line if running on CPU

# Set the particle positions
sim.context.setPositions(modeller.positions)
# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy(tolerance=1*kilojoule/mole, maxIterations=1000)
LocalEnergyMinimizer.minimize(sim.context,tolerance=1*kilojoule/mole,maxIterations=1000)

sim.context.setVelocitiesToTemperature(298.15*kelvin)

sim.reporters.append(DCDReporter('%s', %d))

sim.reporters.append(StateDataReporter(open('%s', 'w'), %d, step=True,
      potentialEnergy=True, totalEnergy=True, temperature=True, density=True,
          progress=True, remainingTime=True, speed=True, separator='\t', totalSteps = %d))

print('Running Production...')
sim.step(%d)
with open('out.chk', 'wb') as f:
      f.write(sim.context.createCheckpoint())
print('Saving pdb file')
positions = sim.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(sim.topology, positions, open('%s', 'w'))
print('Done!')"""

    openmm_script = openmm_script % (str(conc_m)+'m_box.pdb', str(conc_m)+'m_box.pdb', 'nonbonded.xml', d['Box size'], d['Box size'], d['Box size'], cation+'+', cutoff_value, 'out.dcd', d['Steps']['Report'], 'out_file', d['Steps']['Report'], d['Steps']['Simulation'], d['Steps']['Simulation'], 'out.pdb')
    wdir = WORKDIR+'data/'+pot+'/'+str(cutoff_value)+'nm/'+cation.lower()+anion.lower()+'/'+str(conc_m)+'m/'
    file_handle = open(wdir+'run.py', 'w')
    file_handle.write(openmm_script)
    file_handle.close()

In [ ]:
# Flow control: Set aurora to True if you have access. Else: simulation will be run locally.

aurora = True

if aurora:
    for conc_m in d['Molal concentrations']:
        aurora_script="""#!/bin/bash

#SBATCH -p gpu
#SBATCH --exclusive
#SBATCH --gres=gpu:2
#SBATCH --mem-per-cpu=3100
#SBATCH -N 1
#SBATCH -A lu2019-2-15

# job time, change for what your job requires
#SBATCH -t 24:00:00

# job name
#SBATCH -J NaI-sp1

# filenames stdout and stderr - customise, include %j
#SBATCH -o sim.out
#SBATCH -e sim.err

cd $SLURM_SUBMIT_DIR

module add intelcuda
module unload gcc
module load GCC/4.8.4
module load CUDA/10.1.243

python run.py
"""
        wdir = WORKDIR+'data/'+pot+'/'+str(cutoff_value)+'nm/'+'/'+cation.lower()+anion.lower()+'/'+str(conc_m)+'m/'
        with open(wdir+'aurora.sh', 'w') as text_file:
            text_file.write(aurora_script)

In [ ]:
for conc_m in d['Molal concentrations']:
    wdir = WORKDIR+'data/'+pot+'/'+str(cutoff_value)+'nm'+'/'+cation.lower()+anion.lower()+'/'+str(conc_m)+'m/'
    os.chdir(wdir)
    print("Current working directory: ", os.getcwd())
    if aurora:
        !sbatch aurora.sh

    else:
        !python run.py